## Pauta Lab 1

Definición de clase:

In [1]:
import random
import time
from abc import ABC, abstractmethod
from math import sqrt

## CLASES PARA DEFINIR LOS ELEMENTOS DEL MAPA

In [2]:
class Element(ABC):
    
    def __init__(self, x, y):
        self.x= x
        self.y= y
    
class Obstaculo(Element):
    
    def __init__(self, x, y):
        super().__init__(x, y)
        self.pas= False
        self.letter= "X"
    
    def __repr__(self):
        return self.letter
    
class Transitable(Element):
    
    def __init__(self, x, y):
        super().__init__(x, y)
        self.pas= True
        self.letter= "_"
    
    def __repr__(self):
        return self.letter

class Diamond(Element):
    
    def __init__(self, x, y):
        super().__init__(x, y)
        self.pas= True
        self.letter= "D"
        self.points= 5
        
    def points_get(self):
        return self.points
    
    def __repr__(self):
        return self.letter

class Coin(Element):
    
    def __init__(self, x, y):
        super().__init__(x, y)
        self.pas= True
        self.letter= "M"
        self.points= 1
    
    def points_get(self):
        return self.points
    
    def __repr__(self):
        return self.letter

class Chest(Element):
    
    def __init__(self, x, y):
        super().__init__(x, y)
        self.pas= True
        self.letter= "F"
        self.points= [-3, 10]
    
    def points_get(self):
        return random.randint(self.points[0], self.points[1])
    
    def __repr__(self):
        return self.letter

class Speed(Element):

    def __init__(self, x, y):
        super().__init__(x, y)
        self.pas= True
        self.letter= "V"
        self.multip= 2 #multiplica la velocidad a el doble
        self.time= 2 #lo hace por 2 instantes de tiempo
    
    def __repr__(self):
        return self.letter

            
class SafeZone(Element):
    
    def __init__(self, x, y):
        super().__init__(x, y)
        self.pas= True
        self.letter= "Z"
        self.total_points= 0
        
    
    def __repr__(self):
        return self.letter
    

## CLASES DE CARACOLES 

Se crearon 3 tipos de caracoles:
- Caracol_Laser: Al inicio de un tiempo t (antes de moverse) dispara una laser alguna dirección (arriba, abajo, izquierda o derecha) que si le llega a otro caracol, lo destruye.
- Caracol_Safe: Este caracol juega muy seguro, por lo que se salva de la muerte. Tiene "una vida extra" más que el resto de los caracoles. Si lo destruye un termita su vida extra no aplica.
- Caracol_Fast: Este caracol es el más rapido de todos, desesperado por agarrar puntos avanza más rapido que sus pares.

In [3]:
class Caracol():
    
    #Clase padre de los caracoles donde se defininen los movimientos y como seleccionan los objetivos
    
    # se define un id entre las clases para asignar los caracoles
    id= 1
    
    def __init__(self, x, y):
        self.x= x
        self.death= False
        self.y= y
        self.points= 0
        self.letter= "C"
        self.finished= False
        self.movements= 2 #Se mueve a lo más 2 casillas.
        self.objective= False
        self.path= []
        self.id= Caracol.id
        self.life = 1
        self.time_boost= 0
        Caracol.id+= 1
        
    #Esta función se encarga de buscar el cofre, moneda o diamante más cercano a la ubicación actual del caracol. 
    #Luego, esta pasa a ser el objetivo al que se va a mover el caracol
    #Esta distancia no considera los obstaculos
    
    def find_nearby_target(self, mapa, dicc_restore):
        #La prioridad del caracol es la siguiente:
        #Primero va a buscar puntos
        #Sino quedan más puntos para recolectar, procede a buscar la zona segura más cercana desocupada
        #En caso de no haber espacio en la zona segura, el caracol no se movera
        
        nearby = None
        nearby_z = None
        nearby_c = None
        
        #Aqui se revisa si estamos parados en un punto seguró.
                
        for row in mapa:
            for col in row:
                if col.letter in ["D", "M", "F"]:
                    if nearby==None:
                        nearby= col
                    elif abs(nearby.x-self.x)+ abs(nearby.y-self.y)>abs(self.x-col.x)+abs(self.y-col.y):
                        nearby= col
                if col.letter== "Z":
                    if nearby_z==None:
                        nearby_z= col
                    elif abs(nearby_z.x-self.x)+ abs(nearby_z.y-self.y)>abs(self.x-col.x)+abs(self.y-col.y):
                        nearby_z= col

        #Se prioriza según lo mencionado
        if nearby== None:
            if (self.y, self.x)  in dicc_restore:
                if dicc_restore[(self.y, self.x)].letter=="Z":
                    nearby_z= dicc_restore[(self.y, self.x)]
            nearby= nearby_z
            
        self.objective= nearby
    
    
    #Esta función se encarga de encontrar el camino más corto al objetivo del caracol
    def find_path(self, mapa):
        #Si no hay objetivos, no se mueve
        self.path= [len(mapa[0])*len(mapa), []]
        visited = [[0 for x in range(len(mapa[0]))] for y in range(len(mapa))]
        self.min_path(self.x, self.y, visited, [0,[]], mapa)
    
    
    #Se busca el camino más corto utlizando backtraking,probando los caminos y quedandose con el más corto.
    def min_path(self, x, y, visited, path, mapa):
        
        #Condición de termino
        if x==self.objective.x and y==self.objective.y:
            if self.path[0]> path[0]:
                self.path= path.copy()
                self.path[1]= path[1].copy()
                self.path[1].append((x,y))
                self.path[1].pop(0)
            return 
        visited[y][x] = True
        path[0]+=1
        path[1].append((x,y))
        if self.valid_move(x+1, y, mapa, visited):
            self.min_path(x+1, y, visited, path, mapa)
           
        if self.valid_move(x, y+1, mapa, visited):
            self.min_path(x, y+1, visited, path, mapa)
            
        if self.valid_move(x-1, y, mapa, visited):
            self.min_path(x-1, y, visited, path, mapa)
            
        if self.valid_move(x, y-1, mapa, visited):
            self.min_path(x, y-1, visited, path, mapa)
        path[0]-=1
        path[1].pop()
        visited[y][x] = False
        
     
    #Revisa si el movimiento es valido.
    #Que no se salga del mapa o que no pase por un obstaculo y que evite caracoles.
    def valid_move(self, x, y, mapa, visited):
        if x<0 or y<0 or y>=len(mapa) or x>=len(mapa[0]):
            return False
        if mapa[y][x].letter=="X" or mapa[y][x].letter== "C" or visited[y][x]==True:
            return False
        return True
    
    #Se relizan los movimiento del caracol por el camino buscado anteriormente.
    def move(self, mapa):
        
        #Se disminuye el tiempo en boost, si es que el caracol habia pasado por un V en el turno anterior.
        moves= self.movements
        if self.time_boost>0:
            self.time_boost-=1
            print(f"AL CARACOL {self} LE QUEDA {self.time_boost} TIEMPO DE BOOST")
            if self.time_boost==0:
                speed= Speed(0,0)
                self.movements= int(self.movements/speed.multip)
                
        #El caracol no tiene un camino habilitado       
        if self.path[0]==len(mapa[0])*len(mapa):
                return False  
        path=[]
        #El caracol avanza a traves del path optimo.
        for i in range(moves):
            #revisar si el movimiento es valido
            if self.move_valid(mapa):
                coor= self.path[1].pop(0)
                self.x= coor[0]
                self.y= coor[1]
                path.append(coor)
            else:
                break
            if len(self.path[1])==0:
                print(f"EL CARACOL {self} RECORRIO EL SIGUIENTE CAMINO {path}")
                return True
        print(f"EL CARACOL {self} RECORRIO EL SIGUIENTE CAMINO {path}")
        return False
     
    #Se revisa que no pase por casillas que tengan otros caracoles, en ese caso el caracol se detiene y espera
    #El siguiente instante de tiempo para moverse
    def move_valid(self, mapa):
        coor= self.path[1][0]
        if mapa[coor[1]][coor[0]].letter=="C":
            return False
        
        #Se registra el boost de velocidad en caso de pasar por una casilla "V"
        #El boost estara disponible los siguientes turnos
        
        elif mapa[self.path[1][0][1]][self.path[1][0][0]].letter=="V":
            self.movements= self.movements*mapa[self.path[1][0][1]][self.path[1][0][0]].multip
            self.time_boost= mapa[self.path[1][0][1]][self.path[1][0][0]].time
            
            print(f"EL CARACOL {self} TIENE UN BOOST POR {self.time_boost} TIEMPO")
        return True
    
    def hability(self, mapa, caracoles):
        return mapa
    
    #Sirve para ordenar los caracoles por su puntaje, más adelante
    def __lt__(self, other):
         return self.points < other.points
                
    def __repr__(self): 
        return self.letter+ str(self.id)

#CREAMOS LOS DISTINTOS TIPOS DE CARACOLES CON SUS FUNCIONES Y ATRIBUTOS    
class Caracol_Fast(Caracol):
    
    def __init__(self, x, y):
        super().__init__(x, y)
        self.movements= 4
    
    def __repr__(self):
        return self.letter+ str(self.id)
        

class Caracol_Laser(Caracol):
    
    def __init__(self, x, y):
        super().__init__(x, y)
        self.prob= 0.2
        self.life= 1
    
    # Este caracol tiene una probabilidad de disparar un rayo laser en alguna dirección (arriba, abajo, derecha o izquierda)
    # en cada periodo de tiempo. Si un caracol esta en la dirección del rayo laser queda eliminado. Atraviesa obstaculos.
    # Una vez que se instala definitivamente en la zona segura, no puede usar su habilidad.
    
    def hability(self, mapa, caracoles):
        value= random.random()
        if self.prob>value:
            dire= random.choice(["UP", "DOWN", "LEFT", "RIGHT"])
            print(f"EL CARACOL {self} DISPARO SU LASER HACIA {dire}")
            for caracol in caracoles:
                if caracol.death:
                    continue
                if dire=="UP":
                    if caracol.x== self.x and self.y>caracol.y:
                        caracol.finished= True
                        caracol.death= True
                if dire=="DOWN":
                    if caracol.x== self.x and self.y<caracol.y:
                        caracol.finished= True
                        caracol.death= True
                if dire=="LEFT":
                    if caracol.x< self.x and self.y==caracol.y:
                        caracol.finished= True
                        caracol.death= True
                if dire=="RIGHT":
                    if caracol.x> self.x and self.y==caracol.y:
                        caracol.finished= True
                        caracol.death= True
                if caracol.death:
                    if caracol.life==2:
                        caracol.finished= False
                        caracol.death= False
                        print(f"EL CARACOL {caracol} RECIBIÓ EL DISPARO; PERDIÓ UNA VIDA PERO SOBREVIVIÓ")
                    else:
                        mapa[caracol.y][caracol.x]= Transitable(caracol.x, caracol.y)
                        print(f"EL CARACOL {caracol} RECIBIÓ UN DISCARO DE {self} Y murio")
        return mapa
    
    def __repr__(self):
        return self.letter+ str(self.id)
        

class Caracol_Safe(Caracol):
        
    def __init__(self, x, y):
        super().__init__(x, y)
        self.life= 2
    
    def __repr__(self):
        return self.letter+ str(self.id)
        

Eventos:
Se definieron los siguientes:
- PointsChange: Cambia los diamantes por monedas y las monedas por diamantes al inicio de t
- Lightning: Caen una cantidad al azar de rayos en el mapa, que si caen sobre un caracol, lo eliminan. (recordar que el caracol safe tiene 2 vidas)
- EraseBoundery: Algunos obstaculos no estan tambien construidos, por lo que tienen cierta probabilidad de caerse y pueden ser transitables. (cambia "X" por "_")
- SlowMovements: Las termitas sabotean a los caracoles y por cierto tiempo t, todos sus movimientos se ven reducidos a la mitad


In [4]:
class Evento(ABC):
    
    @abstractmethod
    def event_action(self, mapa):
        pass

class PointsChange(Evento):
    
    # Este evento cambia las monedas por diamantes y los diamantes por monedas.
    
    def event_action(self, mapa):
        print(f"EVENTO: PointsChange ACTIVADO")
        
        i= 0
        while i<len(mapa):
            u= 0
            while u<len(mapa[i]):
                if isinstance(mapa[i][u], Coin):
                    mapa[i][u]= Diamond(u, i)
                elif isinstance(mapa[i][u], Diamond):
                    mapa[i][u]= Coin(u, i)
                u+=1
            i+=1
        return mapa
        
class Lightning(Evento):
    
    # Este evento lanza un numero al azar de relampagos, dependiendo del tamaño del mapa, en las casillas del mapa
    # Si un caracol se encuentra en una de las casillas donde cayó un relampago muere. (pueden caer en las zonas seguras)
    
    def event_action(self, mapa, restore_values):
        print(f"EVENTO: Lightning ACTIVADO")
        
        number_lightning= random.randint(1, int(sqrt(len(mapa)*len(mapa))))
        caracoles_death= []
        i=0
        while i<number_lightning:
            x= random.randint(0, len(mapa[0])-1)
            y= random.randint(0, len(mapa)-1)
            
            print(f"CAYO UN RAYO EN x: {x}  y: {y}")
            if mapa[y][x].letter=="C":
                if mapa[y][x].life==2:
                    print(f"EL RAYO CAYÓ SOBRE {mapa[y][x]} PERO COMO TIENE 2 VIDAS ESCAPÓ LA ELIMINACIÓN")
                    mapa[y][x].life-=1
                else:
                    caracoles_death.append(mapa[y][x])
                    print(f"EL RAYO CAYÓ SOBRE {mapa[y][x]} QUITANDOLE LA VIDA")
                    if (y, x) in restore_values:
                        mapa[y][x].finished= True
                        mapa[y][x].death= True
                        mapa[y][x]= restore_values[(y,x)]
                    else:
                        mapa[y][x].finished= True
                        mapa[y][x].death= True
                        mapa[y][x]= Transitable(x, y)
            i+=1
        
        #Se retorna el mapa con los caracoles eliminados y los caracoles muertos.
        return mapa, caracoles_death 

class EraseBoundery(Evento):
    
    
    def event_action(self, mapa):
        print(f"EVENTO: EraseBoundery ACTIVADO")
        
        #Revisamos la cantidad de obstaculos
        count_obs= 0
        i= 0
        lista_obs= []
        while i<len(mapa):
            u= 0
            while u<len(mapa[i]):
                if isinstance(mapa[i][u], Obstaculo):
                    lista_obs.append(mapa[i][u])
                    count_obs+= 1 
                u+=1
            i+=1 
                                             
        #Un numero al azar de obstaculos que se destruyen
        if count_obs==0:
            print(f"YA NO QUEDAN OBSTACULOS POR ELIMINAR")
            return mapa
        number_destroyed= random.randint(1, int(sqrt(count_obs)))
        obstaculos= random.sample(lista_obs, number_destroyed)
        for obstaculo in obstaculos:
            x= obstaculo.x
            y= obstaculo.y
            print(f"SE ELIMINO EL OBSTACULO EN x: {x}  y: {y}")
            mapa[y][x]= Transitable(x, y)
        return mapa
    
class SlowMovements(Evento):
    
    def __init__(self):
        self.time= 4
        self.actual_time= 4
    
    
    def event_action(self, caracoles):
        print(f"EVENTO: {type(self)} ACTIVADO LA VELOCIDAD DE LOS CARACOLES DISMINUYÓ A LA MITAD POR t={self.time}")
        
        if self.time==self.actual_time:
            for caracol in caracoles:
                caracol.speed= int(caracol.speed/2)
        return 
    
    #REVISA CUANDO TIEMPO A PASADO PARA QUE TERMINE EL EFECTO DEL EVENTO.
    #Este evento no puede ocurrir nuevamente si el efecto anterior aun no termina
    def update_time(self):
        if self.time==self.actual_time:
            self.actual_time=-1
            print(f"EVENTO {type(self)}: QUEDAN t={self.actual_time} PARA QUE LOS CARACOLES RECUPEREN SU VELOCIDAD")
            if self.actual_time==0:
                self.actual_time= self.time
                return True
            else: 
                return False
        return False

## CLASE MAPA:

Aqui se registra todo lo ocurrido en el juego y se simulan los eventos a ocurrir

In [6]:
class Map():

    def __init__(self, mapa, n_caracoles):
        self.events=[PointsChange(), EraseBoundery(), Lightning(), SlowMovements()] 
        self.dicc_restore_values={}
        self.mapa= mapa
        self.n_caracoles= n_caracoles
        self.time= 30    
        self.caracoles= []
        self.caracoles_death= []
        self.assign_elements()
        self.prob=0.2
        
        
        
    def assign_elements(self):
        i=0
        while i<len(self.mapa):
            u=0
            while u<len(self.mapa[i]):
                if self.mapa[i][u]=="D":
                    self.mapa[i][u]= Diamond(u,i)
                elif self.mapa[i][u]=="V":
                    self.mapa[i][u]= Speed(u,i)
                    self.dicc_restore_values[(i,u)]= self.mapa[i][u]
                elif self.mapa[i][u]=="M":
                    self.mapa[i][u]= Coin(u,i)
                elif self.mapa[i][u]=="Z":
                    self.mapa[i][u]= SafeZone(u,i)
                    self.dicc_restore_values[(i,u)]= self.mapa[i][u]
                elif self.mapa[i][u]=="F":
                    self.mapa[i][u]= Chest(u,i)
                elif self.mapa[i][u]== "_":
                    self.mapa[i][u]= Transitable(u,i)
                elif self.mapa[i][u]== "X":
                    self.mapa[i][u]= Obstaculo(u,i)
                u+=1
            i+=1
        self.assign_caracoles()  
        self.view_map()
        
        
    def assign_caracoles(self):
        n_c=0
        Caracol.id=1
        print("LOS CARACOLES QUE COMPITEN SON:")
        while n_c<self.n_caracoles:
            row= random.randint(0, len(self.mapa)-1)
            col= random.randint(0, len(self.mapa[row])-1)
            if self.mapa[row][col].letter == "_":  #revisamos que sea una casilla vacia:
                caracol= random.randint(1,3)
                if caracol== 1:
                    caracol= Caracol_Safe(col, row)
                elif caracol== 2:
                    caracol=  Caracol_Laser(col, row)
                elif caracol== 3:
                    caracol=  Caracol_Fast(col, row)
                self.mapa[row][col]= caracol
                print(f"{caracol} de la clase {type(caracol).__name__}")
            
                self.caracoles.append(caracol)
                n_c+=1      
    
    def view_map(self):
        print("")
        print("-----------------------------------------------")
        for row in self.mapa:
            print(row)
        print("-----------------------------------------------")
        
        
    def simulate(self):
        t=0
        while self.check_end():
            print(f"INICIO DE LAS ACCCIONES DEL TIEMPO T={t}")
            
            #Avance de termitas:
            
            print(f"EVENTOS ACTIVADOS")
            prob= random.random()
            
            #Evento Points Change
            if prob<0.2:
                self.mapa= self.events[0].event_action(self.mapa)

            #Evento Lightning
            prob= random.random()
            if prob<0.2:
                self.mapa, caracoles_death= self.events[2].event_action(self.mapa, self.dicc_restore_values)
                self.caracoles_death.extend(caracoles_death) 
                
            #Evento EraseBoundery
            prob= random.random()
            if prob<0.2:
                self.mapa= self.events[1].event_action(self.mapa)
            
            
            #Movimientos de los caracoles:
            print("\nMOVIMIENTOS CARACOLES")
            caracoles= random.sample(self.caracoles, len(self.caracoles))
            for caracol in caracoles:
                
                #Si el caracol fue eliminado o llego a una zona segura
                if caracol.finished:
                    continue
                caracol.find_nearby_target(self.mapa, self.dicc_restore_values)
                
                #activamos las habilidades de los caracoles en juego
                self.mapa = caracol.hability(self.mapa, self.caracoles)
                #Si no hay un camino, ya que no hay objetivos, se sigue con el siguiente caracol.
                if caracol.objective== None:
                    continue
                caracol.find_path(self.mapa)
                
                #Si el caracol que vamos a mover estaba parado en Z o V, restauramos estos valores
                if (caracol.y, caracol.x) in self.dicc_restore_values:
                    self.mapa[caracol.y][caracol.x]= self.dicc_restore_values[(caracol.y,caracol.x)]
                #Sino, el valor es "_"
                else:
                    self.mapa[caracol.y][caracol.x]= Transitable(caracol.x, caracol.y)
                if caracol.move(self.mapa):
                    
                    #Si su destino era una zona segura y llego, termino su rol en el juego.
                    if self.mapa[caracol.y][caracol.x].letter=="Z":
                        caracol.finished= True
                        
                        print(f"EL CARACOL {caracol} LLEGO A LA ZONA SEGURA PARA QUEDARSE")
                        self.caracoles[caracol.id-1].finished= True
                    else:
                        puntos= self.mapa[caracol.y][caracol.x].points_get()
                        self.caracoles[caracol.id-1].points +=  puntos
                        print(f"EL CARACOL {caracol} GANO {puntos} DE UN/A {type(self.mapa[caracol.y][caracol.x]).__name__} PARA UN PUNTAJE TOTAL DE: {self.caracoles[caracol.id-1].points}")
                self.mapa[caracol.y][caracol.x]= caracol
            print("\n\nMAPA PRE TERMITAS")
            self.view_map()
            print("AVANCE DE LAS TERMITAS \n")
            self.termitas()
            self.fix_pos()
            t+=1
        self.final_table()
    
    def final_table(self):
        #Seleccionamos solo los vivos
        
        caracoles_alive= [] 
        caracoles_death= []
        for caracol in self.caracoles:
            if caracol.death:
                caracoles_death.append(caracol)
            else:
                caracoles_alive.append(caracol)
                
        #Los ordenamos
        caracoles_alive.sort(reverse=True)
    
        print("TABLA FINAL CON LOS GANADORES")
        print("---------------------------------------------------------------")
        print("LOS CARACOLES QUE SOBREVIVIERON ORDENADOS POR MEJOR PUNTAJE SON")
        print("---------------------------------------------------------------")
        for caracol in caracoles_alive:
            print(f"CARACOL {caracol} con un puntaje de {caracol.points}")
            
        
    
    def termitas(self):
        dire= random.choice(["UP", "DOWN", "LEFT", "RIGHT"])

        #Se elimina el lado del mapa y se eliminan los caracoles que estaban en esos lados :(
        #En cada turno las termitas intentan comerse un lado. A veces tienen exito, a veces no.
        #Nunca se podran comer un borde que contenga una "Z"

        if dire=="UP":
            #Revisamos que no se coman la zona segura
            if "Z" in list(map(lambda x: x.letter, self.mapa[0])):
                return False
            for element in self.mapa[0]:
                if element.letter=="C":
                    if element.finished:
                        return False
                    
            print("LA TERMITAS SE COMIERON LA PARTE SUPERIOR DEL MAPA")
            for element in self.mapa[0]:
                
                #Eliminamos los caracoles que se comieron las termitas
                if element.letter=="C":
                    self.caracoles[element.id-1].death= True
                    self.caracoles[element.id-1].finished= True
                    print(f"EL caracol {self.caracoles[element.id-1]} murió comido por las terminas")
            self.mapa.pop(0)
        elif dire=="DOWN":
            
            #Revisamos que no se coman la zona segura
            if "Z" in list(map(lambda x: x.letter, self.mapa[len(self.mapa)-1])):
                return False
            for element in self.mapa[-1]:
                if element.letter=="C":
                    if element.finished:
                        return False
                    
                    
            print("LA TERMITAS SE COMIERON LA PARTE INFERIOR DEL MAPA")
            for element in self.mapa[-1]:
                
                #Eliminamos los caracoles que se comieron las termitas
                if element.letter=="C":
                    self.caracoles[element.id-1].death= True
                    self.caracoles[element.id-1].finished= True
                    print(f"EL caracol {self.caracoles[element.id-1]} murió comido por las terminas")   
            self.mapa.pop()
        elif dire in ["LEFT", "RIGHT"]:
            print(f"LA TERMITAS SE COMIERON LA PARTE {dire} DEL MAPA")
            i=0
            
            #Revisamos que no se coman la zona segura
            if dire=="LEFT":
                if self.check_z_left(self.mapa):
                        return False
            if dire=="RIGHT":
                if self.check_z_right(self.mapa):
                        return False
            while i<len(self.mapa):
                if dire=="LEFT":
                    
                    
                    #Eliminamos los caracoles que se comieron las termitas
                    if self.mapa[i][0].letter== "C":
                        self.caracoles[self.mapa[i][0].id-1].death= True
                        self.caracoles[self.mapa[i][0].id-1].finished= True
                        print(f"EL caracol {self.caracoles[self.mapa[i][0].id-1]} murió comido por las terminas")  
                    self.mapa[i].pop(0)
                elif dire=="RIGHT":
                    
                    #Eliminamos los caracoles que se comieron las termitas
                    if self.mapa[i][-1].letter== "C":
                        self.caracoles[self.mapa[i][-1].id-1].death= True
                        self.caracoles[self.mapa[i][-1].id-1].finished= True
                        print(f"EL caracol {self.caracoles[self.mapa[i][-1].id-1]} murió comido por las terminas") 
                    self.mapa[i].pop()
                i+=1
        #Restauramos los valores guardados en nuestro diccionario, los valores de x e y y las keys
        
        print("MAPA POST TERMITAS")
        self.restore_values(dire)
        return 
    
    #Restauramos los valores guardados en nuestro diccionario, los valores de x e y y las keys
    def restore_values(self, dire):
        keys_to_delete= []
        keys_values_add= []
        for key in self.dicc_restore_values:
            if dire=="UP":
                if key[1]>0:
                    value= self.dicc_restore_values[key]
                    keys_to_delete.append(key)
                    key_new= (key[0]-1, key[1])
                    value.y= key[0]-1
                    keys_values_add.append([key_new, value])
                else:
                    keys_to_delete.append(key)
            elif dire=="LEFT":
                if key[0]>0:
                    value= self.dicc_restore_values[key]
                    keys_to_delete.append(key)
                    key_new= (key[0], key[1]-1)
                    value.x= key[1]-1
                    keys_values_add.append([key_new, value])
                else:
                    keys_to_delete.append(key)
        for key in keys_to_delete:
            del self.dicc_restore_values[key]
            
        for key in keys_values_add:
            self.dicc_restore_values[key[0]]=key[1]
            
                    
    
    #Revisamos que no se coman la zona segura
    def check_z_left(self, mapa):
        for row in mapa:
            if row[0].letter== "Z":
                return True
            elif row[0].letter=="C":
                if row[0].finished==True:
                    return True
        return False
    
    #Revisamos que no se coman la zona segura
    def check_z_right(self, mapa):
        for row in mapa:
            if row[-1].letter== "Z":
                return True
            elif row[-1].letter=="C":
                if row[-1].finished==True:
                    return True
        return False
        
    #Como cambio el tamaño del mapa, actulizamos las posiciones x e y de los elementos y caracoles
    def fix_pos(self):
        i=0
        while i<len(self.mapa):
            u=0
            while u<len(self.mapa[i]):
                if self.mapa[i][u]=="C":
                    self.caracoles[self.mapa[i][u].id-1].x= u
                    self.caracoles[self.mapa[i][u].id-1].y= i
                self.mapa[i][u].x= u
                self.mapa[i][u].y= i  
                u+=1
            i+=1 
        self.view_map()
    
    #revisamos si termino el juego
    def check_end(self):
        for row in self.mapa:
            for element in row:
                if element.letter not in ["Z", "C"]:
                    return True
                else:
                    if element.letter=="C":
                        if self.caracoles[element.id-1].finished== False:
                            return True
                    else:
                        continue
        return False
                    
            
        
        
        

        

Input

In [7]:
ma= [ [ 'F' , '_' , '_' , '_' , '_' , 'X' , 'M' ] , [ '_' , '_' , '_' , '_' , 'F' , 'X' , '_' ] , [ 'X' , 'X' , 'X' , 'Z' , '_' , 'X' , '_' ] , [ '_' , 'F' , '_' , 'Z' , '_' , 'X' , '_' ] , [ '_' , '_' , '_' , 'Z' , '_' , 'X' , '_' ] , [ '_' , 'X' , '_' , '_' , '_' , '_' , '_' ] , [ 'D' , 'X' , '_' , 'V' , '_' , '_' , 'M' ] ]

In [8]:
simulacion= Map(ma, 5) #numero de caracoles (para probar la simulación bien, resetear el kernel)

LOS CARACOLES QUE COMPITEN SON:
C1 de la clase Caracol_Laser
C2 de la clase Caracol_Fast
C3 de la clase Caracol_Safe
C4 de la clase Caracol_Safe
C5 de la clase Caracol_Laser

-----------------------------------------------
[F, _, _, C5, C4, X, M]
[_, _, _, _, F, X, C3]
[X, X, X, Z, _, X, _]
[C2, F, _, Z, _, X, _]
[_, _, _, Z, _, X, _]
[_, X, _, C1, _, _, _]
[D, X, _, V, _, _, M]
-----------------------------------------------


In [9]:
simulacion.simulate()

INICIO DE LAS ACCCIONES DEL TIEMPO T=0
EVENTOS ACTIVADOS
EVENTO: EraseBoundery ACTIVADO
SE ELIMINO EL OBSTACULO EN x: 1  y: 6
SE ELIMINO EL OBSTACULO EN x: 5  y: 3
SE ELIMINO EL OBSTACULO EN x: 5  y: 2

MOVIMIENTOS CARACOLES
EL CARACOL C2 RECORRIO EL SIGUIENTE CAMINO [(1, 3)]
EL CARACOL C2 GANO 1 DE UN/A Chest PARA UN PUNTAJE TOTAL DE: 1
EL CARACOL C4 RECORRIO EL SIGUIENTE CAMINO [(4, 1)]
EL CARACOL C4 GANO 4 DE UN/A Chest PARA UN PUNTAJE TOTAL DE: 4
EL CARACOL C1 TIENE UN BOOST POR 2 TIEMPO
EL CARACOL C1 RECORRIO EL SIGUIENTE CAMINO [(3, 6), (2, 6)]
EL CARACOL C5 RECORRIO EL SIGUIENTE CAMINO [(2, 0), (1, 0)]
EL CARACOL C3 RECORRIO EL SIGUIENTE CAMINO [(6, 0)]
EL CARACOL C3 GANO 1 DE UN/A Coin PARA UN PUNTAJE TOTAL DE: 1


MAPA PRE TERMITAS

-----------------------------------------------
[F, C5, _, _, _, X, C3]
[_, _, _, _, C4, X, _]
[X, X, X, Z, _, _, _]
[_, C2, _, Z, _, _, _]
[_, _, _, Z, _, X, _]
[_, X, _, _, _, _, _]
[D, _, C1, V, _, _, M]
-----------------------------------------